![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# Memory Agent Example
The following example shows how to build an agent that uses multiple forms of memory:
  1. Short-term memory (messages in the current conversation)
  2. Long-term memory
     1. Semantic: General knowledge
     2. Episodic: User specific experiences
     3. Procedural: How to do things

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/agents/03_memory_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Packages

In [9]:
%pip install -q langchain-openai langgraph-checkpoint langgraph-checkpoint-redis "langchain-community>=0.2.11" tavily-python langchain-redis



ERROR: Cannot install langgraph-checkpoint and langgraph-checkpoint-redis==0.0.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### OPEN_AI_API key

You must add an OpenAI API key with billing information enabled is required for this lesson.

In [5]:
# NBVAL_SKIP
import os
import getpass



def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")

## Run redis
TODO

NOTE: The existing magic shell commands don't work when run locally for me.

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

## Test connection

In [7]:
import os
from redis import Redis

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")

client = Redis.from_url(REDIS_URL)
client.ping()

True

In [ ]:
import os
from typing import List, Optional
from datetime import datetime
from enum import Enum
from langgraph.graph.message import MessagesState, RemoveMessage
from pydantic import BaseModel, Field
from queue import Queue

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.globals import set_llm_cache


from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt.chat_agent_executor import create_react_agent
from langgraph.checkpoint.redis import RedisSaver
from langchain_redis import RedisCache

import redis
from redisvl.index import SearchIndex
from redisvl.query import VectorRangeQuery
from redisvl.query.filter import Tag
from redisvl.schema.schema import IndexSchema
from redisvl.query import FilterQuery


from util import get_logger
from dotenv import load_dotenv
import threading
import json
import time
import ulid

load_dotenv()

logger = get_logger(__name__)


# Memory types
class MemoryType(str, Enum):
    EPISODIC = "episodic"  # User specific experiences
    PROCEDURAL = "procedural"  # How to do things
    SEMANTIC = "semantic"  # General knowledge


# Redis Configuration
REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = int(os.getenv("REDIS_PORT", 6379))
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
VECTOR_DIM = 1536  # ada-002 has 1536 dimensions

MESSAGE_SUMMARIZATION_THRESHOLD = 12

# Models and Tools
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
summarizer = ChatOpenAI(model="gpt-4o", temperature=0.3)
web_search_tool = TavilySearchResults(max_results=2)
openai_embed = OpenAIEmbeddings(model="text-embedding-ada-002")


class RuntimeState(MessagesState):
    """Agent state (just messages for now)"""

    pass


class Memory(BaseModel):
    content: str
    type: MemoryType
    metadata: str


class Memories(BaseModel):
    memories: List[Memory]


class StoredMemory(Memory):
    id: str  # The redis key
    memory_id: ulid.ULID = Field(default_factory=lambda: ulid.ULID())
    created_at: datetime = Field(default_factory=datetime.now)
    user_id: str


class StoredMemories(BaseModel):
    memories: List[StoredMemory]


# Initialize Redis connection
redis_client = redis.Redis(
    host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD, decode_responses=True
)

# Define schema for memory index
memory_schema = IndexSchema(
    **{
        "index": {
            "name": "agent_memories",
            "prefix": "memory:",
            "key_separator": ":",
            "storage_type": "json",
        },
        "fields": [
            {"name": "content", "type": "text"},
            {"name": "memory_type", "type": "tag"},
            {"name": "metadata", "type": "text"},
            {"name": "created_at", "type": "text"},
            {"name": "user_id", "type": "tag"},
            {"name": "memory_id", "type": "tag"},
            {
                "name": "embedding",
                "type": "vector",
                "attrs": {
                    "algorithm": "flat",
                    "dims": VECTOR_DIM,
                    "distance_metric": "cosine",
                    "datatype": "float32",
                },
            },
        ],
    }
)

# Create search index
try:
    memory_index = SearchIndex(
        schema=memory_schema, redis_client=redis_client, overwrite=True
    )
    memory_index.create()
    logger.info("Memory index ready")
except Exception as e:
    logger.error(f"Error creating index: {e}")
    raise e


cache = RedisCache(
    redis_client=redis_client,
)
set_llm_cache(cache)

# Create a checkpoint saver for LangGraph short-term memory
redis_saver = RedisSaver(redis_client=redis_client)
redis_saver.setup()

# Create agents with specific roles
travel_agent = create_react_agent(
    model=llm,
    tools=[web_search_tool],
    checkpointer=redis_saver,
    prompt=SystemMessage(
        content="""
        You are a travel assistant helping users plan their trips. You remember user preferences
        and provide personalized recommendations based on past interactions.
        
        You have access to the following types of memory:
        1. Short-term memory: The current conversation thread
        2. Long-term memory: 
           - Episodic: User preferences and past trip experiences
           - Procedural: How to book flights, hotels, etc.
           - Semantic: General knowledge about travel destinations
           
        Always be helpful, personal, and context-aware in your responses.
        """
    ),
)

# TODO: Could be a function instead
memory_manager = create_react_agent(
    model=llm,
    tools=[],
    checkpointer=redis_saver,
    response_format=Memories,
    prompt=SystemMessage(
        content="""
        You are a memory management assistant that helps extract important information from 
        conversations. Your job is to identify what information should be stored in long-term 
        memory from the conversation history.
        
        For each piece of information, determine:
        1. Whether it should be stored as episodic, procedural, or semantic memory
        2. What metadata should be attached to it
        
        Reply with a JSON-formatted list of memories to store. Example:
        ```
        [
          {
            "content": "User prefers window seats on flights",
            "type": "episodic",
            "metadata": {"category": "flight_preference", "importance": "high"}
          },
          {
            "content": "Steps to book a flight with layover preferences",
            "type": "procedural",
            "metadata": {"category": "booking_process"}
          }
        ]
        ```
        
        Only extract information that would be useful in future conversations.
        """
    ),
)


def similar_memory_exists(
    content: str,
    memory_type: MemoryType,
    user_id: str,
    similarity_threshold: float = 0.2,
) -> bool:
    """Check if a similar memory already exists in the database"""
    # Create embedding for the new content
    query_embedding = openai_embed.embed_query(content)

    of_type_for_user = (Tag("user_id") == user_id) & (Tag("memory_type") == memory_type)

    # Search for similar memories
    vector_query = VectorRangeQuery(
        vector=query_embedding,
        num_results=1,
        vector_field_name="embedding",
        filter_expression=of_type_for_user,
        distance_threshold=similarity_threshold,
        return_fields=["content"],
    )
    results = memory_index.query(vector_query)
    logger.debug(f"Similar memory search results: {results}")

    if results:
        logger.debug(f"Similar memory found: {results[0]['id']}")
        logger.info(f"Similar memory found, skipping storage: {content}")
        return True

    return False


def store_memory(
    content: str,
    memory_type: MemoryType,
    user_id: str,
    metadata: Optional[str] = None,
):
    """Store a memory in Redis, avoiding duplicates"""
    if metadata is None:
        metadata = "{}"

    logger.info(f"Preparing to store memory: {content}")

    if similar_memory_exists(content, memory_type, user_id):
        logger.info("Similar memory found, skipping storage")
        return

    embedding = openai_embed.embed_query(content)

    memory_data = {
        "user_id": user_id,
        "content": content,
        "memory_type": memory_type.value,
        "metadata": metadata,
        "created_at": datetime.now().isoformat(),
        "embedding": embedding,
        "memory_id": str(ulid.ULID()),
    }

    # Store in Redis
    try:
        memory_index.load([memory_data])
    except Exception as e:
        logger.error(f"Error storing memory: {e}")
        return

    logger.info(f"Stored {memory_type} memory: {content}")


def retrieve_memories(
    query: str, memory_type: Optional[MemoryType] = None, limit: int = 5
) -> StoredMemories:
    """Retrieve relevant memories from Redis"""
    # Create vector query
    logger.debug(f"Retrieving memories for query: {query}")
    vector_query = VectorRangeQuery(
        vector=openai_embed.embed_query(query),
        return_fields=["content", "memory_type", "metadata", "created_at"],
        num_results=limit,
        vector_field_name="embedding",
        distance_threshold=0.2,
    )

    # Add filter for memory type if specified
    if memory_type:
        vector_query.set_filter(f"@memory_type:{{{memory_type}}}")

    # Execute search
    results = memory_index.query(vector_query)

    # Parse results
    memories = []
    for doc in results:
        try:
            memory = StoredMemory(
                id=doc["id"],
                memory_id=doc["memory_id"],
                user_id=doc["user_id"],
                content=doc["content"],
                type=MemoryType(doc["memory_type"]),
                created_at=doc["created_at"],
                metadata=doc["metadata"],
            )
            memories.append(memory)
        except Exception as e:
            logger.error(f"Error parsing memory: {e}")
            continue
    return StoredMemories(memories=memories)


def extract_memories(
    last_processed_message_id: Optional[str],
    state: RuntimeState,
    config: RunnableConfig,
) -> Optional[str]:
    """Extract and store memories in long-term memory"""
    if len(state["messages"]) < 3:  # Need at least a user message and agent response
        return last_processed_message_id

    user_id = config.get("configurable", {}).get("user_id", None)
    if not user_id:
        logger.warning("No user ID found in config when extracting memories")
        return last_processed_message_id

    # Get the messages
    messages = state["messages"]

    # Find the newest message ID (or None if no IDs)
    newest_message_id = None
    for msg in reversed(messages):
        if hasattr(msg, "id") and msg.id:
            newest_message_id = msg.id
            break

    # If we've already processed up to this message ID, skip
    if (
        last_processed_message_id
        and newest_message_id
        and last_processed_message_id == newest_message_id
    ):
        logger.debug(f"Already processed messages up to ID {newest_message_id}")
        return last_processed_message_id

    # Get the last few messages for context
    recent_messages = messages[-5:] if len(messages) > 5 else messages

    # Format messages for the memory agent
    message_history = "\n".join(
        [
            f"{'User' if isinstance(msg, HumanMessage) else 'Assistant'}: {msg.content}"
            for msg in recent_messages
        ]
    )

    # Ask memory agent to extract memories
    prompt = f"""
    Please analyze this recent conversation and extract important information that 
    should be stored in long-term memory:
    
    {message_history}
    
    What information should be stored in long-term memory?
    """

    result = memory_manager.invoke(
        {"messages": [HumanMessage(content=prompt)]}, config=config
    )
    memories_to_store: Memories = result["structured_response"]

    # Store each extracted memory
    for memory_data in memories_to_store.memories:
        store_memory(
            content=memory_data.content,
            memory_type=memory_data.type,
            user_id=user_id,
            metadata=memory_data.metadata,
        )

    # Return data with the newest processed message ID
    return newest_message_id


def retrieve_relevant_memories(
    state: RuntimeState, config: RunnableConfig
) -> RuntimeState:
    """Retrieve relevant memories based on the current conversation"""
    if not state["messages"]:
        logger.debug("No messages in state")
        return state

    logger.debug(f"inbound retrieve_relevant_memories: {len(state['messages'])}")

    latest_message = state["messages"][-1]
    if not isinstance(latest_message, HumanMessage):
        logger.debug("Latest message is not a HumanMessage: ", latest_message)
        return state

    query = str(latest_message.content)
    episodic = retrieve_memories(query=query, memory_type=MemoryType.EPISODIC, limit=3)
    procedural = retrieve_memories(
        query=query, memory_type=MemoryType.PROCEDURAL, limit=2
    )
    semantic = retrieve_memories(query=query, memory_type=MemoryType.SEMANTIC, limit=2)
    relevant_memories = episodic.memories + procedural.memories + semantic.memories

    logger.debug(f"All relevant memories: {relevant_memories}")

    if relevant_memories:
        memory_context = "\n\n### Relevant memories from previous conversations:\n"

        # Group by memory type
        memory_types = {
            MemoryType.EPISODIC: "User Preferences & History",
            MemoryType.PROCEDURAL: "Booking Procedures",
            MemoryType.SEMANTIC: "Destination Knowledge",
        }

        for mem_type, type_label in memory_types.items():
            memories_of_type = [m for m in relevant_memories if m.type == mem_type]
            if memories_of_type:
                memory_context += f"\n**{type_label}**:\n"
                for mem in memories_of_type:
                    memory_context += f"- {mem.content}\n"

        augmented_message = HumanMessage(content=f"{query}\n{memory_context}")
        state["messages"][-1] = augmented_message

        logger.debug(f"Augmented message: {augmented_message.content}")

    logger.debug(f"outbound retrieve_relevant_memories: {len(state['messages'])}")

    return state.copy()


def summarize_conversation(
    state: RuntimeState, config: RunnableConfig
) -> Optional[RuntimeState]:
    """
    Summarize a list of messages into a concise summary
    to reduce context length while preserving important information.
    """
    messages = state["messages"]
    current_message_count = len(messages)
    if current_message_count <= MESSAGE_SUMMARIZATION_THRESHOLD:
        logger.debug(f"Not summarizing conversation: {current_message_count}")
        return None

    # Summarize all but the latest message
    messages_to_summarize = state["messages"][:-1]

    # Create a system prompt for the summarizer
    system_prompt = """
    You are a conversation summarizer. Your task is to create a concise summary 
    of the previous conversation between a user and a travel assistant.
    
    The summary should:
    1. Highlight key topics, preferences, and decisions
    2. Include any specific trip details (destinations, dates, preferences)
    3. Note any outstanding questions or topics that need follow-up
    4. Be concise but informative
    
    Format your summary as a brief narrative paragraph.
    """

    # Invoke the summarizer
    summary_messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(
            content=f"Please summarize this conversation:\n\n{messages_to_summarize}"
        ),
    ]

    summary_response = summarizer.invoke(summary_messages, config=config)

    logger.info(f"Summarized {len(messages)} messages into a conversation summary")

    summary_message = SystemMessage(
        content=f"""
        Summary of the conversation so far:
        
        {summary_response.content}
            
            Please continue the conversation based on this summary and the recent messages.
        """
    )
    remove_messages = [
        RemoveMessage(id=msg.id) for msg in messages_to_summarize if msg.id is not None
    ]

    state["messages"] = [  # type: ignore
        *remove_messages,
        summary_message,
        state["messages"][-1],
    ]

    return state.copy()


def respond_to_user(state: RuntimeState, config: RunnableConfig) -> RuntimeState:
    """Generate a response to the user based on the conversation and memories"""
    if not state["messages"]:
        return state

    # Invoke the travel agent with the context messages
    result = travel_agent.invoke({"messages": state["messages"]}, config=config)
    result_messages = result.get("messages", [])

    if result_messages and any(isinstance(m, AIMessage) for m in result_messages):
        # Find the last AI message in the result
        ai_messages = [m for m in result_messages if isinstance(m, AIMessage)]
        if ai_messages:
            agent_response = ai_messages[-1]  # Get the last AI message
            state["messages"].append(agent_response)
        else:
            logger.error("No AIMessage found in result messages")
            agent_response = AIMessage(
                content="I'm sorry, I couldn't understand your request."
            )
    else:
        logger.error("No valid assistant response found in result")
        agent_response = AIMessage(
            content="I'm sorry, I couldn't process your request properly."
        )

    state["messages"].append(agent_response)

    logger.debug(f"respond_to_user: Returning {len(state['messages'])} messages")

    return state.copy()


def memory_worker(memory_queue: Queue, user_id: str):
    """Worker function that processes memory extraction requests from a queue"""
    key = f"memory_worker:{user_id}:last_processed_message_id"

    last_processed_message_id = redis_client.get(key)
    logger.debug(f"Last processed message ID: {last_processed_message_id}")
    last_processed_message_id = (
        str(last_processed_message_id) if last_processed_message_id else None
    )

    while True:
        try:
            # Get the next state and config from the queue (blocks until an item is available)
            state, config = memory_queue.get()

            # Process the memory extraction
            last_processed_message_id = extract_memories(
                last_processed_message_id, state, config
            )
            logger.debug(
                f"Memory worker extracted memories. Last processed message ID: {last_processed_message_id}"
            )

            if last_processed_message_id:
                logger.debug(
                    f"Setting last processed message ID: {last_processed_message_id}"
                )
                redis_client.set(key, last_processed_message_id)

            # Mark the task as done
            memory_queue.task_done()
            logger.debug("Memory extraction completed for queue item")
        except Exception as e:
            logger.exception(f"Error in memory worker thread: {e}")


def consolidate_memories(user_id: str):
    """
    Periodically scan the memory database and merge similar memories.
    This should be run as a scheduled task.
    """
    logger.info(f"Starting memory consolidation for user {user_id}")

    # First, get all memories for the user
    try:
        # For each memory type, consolidate separately
        for memory_type in MemoryType:
            # Get all memories of this type for the user
            of_type_for_user = (Tag("user_id") == user_id) & (
                Tag("memory_type") == memory_type
            )
            filter_query = FilterQuery(filter_expression=of_type_for_user)
            all_memories = memory_index.query(filter_query)
            if not all_memories:
                continue

            # Group similar memories
            processed_ids = set()
            for memory in all_memories:
                if memory["id"] in processed_ids:
                    continue

                memory_embedding = memory["embedding"]
                vector_query = VectorRangeQuery(
                    vector=memory_embedding,
                    num_results=10,
                    vector_field_name="embedding",
                    filter_expression=of_type_for_user
                    & (Tag("memory_id") != memory["memory_id"]),
                    distance_threshold=0.2,
                    return_fields=[
                        "content",
                        "memory_type",
                        "metadata",
                        "user_id",
                        "memory_id",
                    ],
                )
                similar_memories = memory_index.query(vector_query)

                # If we found similar memories, consolidate them
                if similar_memories:
                    combined_content = memory["content"]
                    combined_metadata = memory["metadata"]

                    if combined_metadata:
                        try:
                            combined_metadata = json.loads(combined_metadata)
                        except Exception as e:
                            logger.error(f"Error parsing metadata: {e}")
                            combined_metadata = {}

                    for similar in similar_memories:
                        # Merge the content of similar memories
                        combined_content += f" {similar['content']}"

                        if similar["metadata"]:
                            try:
                                similar_metadata = json.loads(similar["metadata"])
                            except Exception as e:
                                logger.error(f"Error parsing metadata: {e}")
                            similar_metadata = {}

                        combined_metadata = {**combined_metadata, **similar_metadata}

                    # Create a consolidated memory
                    new_metadata = {
                        "consolidated": True,
                        "source_count": len(similar_memories) + 1,
                        **combined_metadata,
                    }
                    consolidated_memory = {
                        "content": summarize_memories(combined_content, memory_type),
                        "memory_type": memory_type.value,
                        "metadata": json.dumps(new_metadata),
                        "user_id": user_id,
                    }

                    # Delete the old memories
                    delete_memory(memory["id"])
                    for similar in similar_memories:
                        delete_memory(similar["id"])

                    # Store the new consolidated memory
                    store_memory(
                        content=consolidated_memory["content"],
                        memory_type=memory_type,
                        user_id=user_id,
                        metadata=consolidated_memory["metadata"],
                    )

                    logger.info(
                        f"Consolidated {len(similar_memories) + 1} memories into one"
                    )

    except Exception as e:
        raise e
        # logger.error(f"Error during memory consolidation: {e}")


def delete_memory(memory_id: str):
    """Delete a memory from Redis"""
    try:
        result = memory_index.drop_keys([memory_id])
    except Exception as e:
        logger.error(f"Deleting memory {memory_id} failed: {e}")
    if result == 0:
        logger.debug(f"Deleting memory {memory_id} failed: memory not found")
    else:
        logger.info(f"Deleted memory {memory_id}")


def summarize_memories(combined_content: str, memory_type: MemoryType) -> str:
    """Use the LLM to create a concise summary of similar memories"""
    try:
        system_prompt = f"""
        You are a memory consolidation assistant. Your task is to create a single, 
        concise memory from these similar memory fragments. This is a {memory_type.value} memory.
        
        Combine the information without repetition while preserving all important details.
        """

        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(
                content=f"Please consolidate these similar memories into one:\n\n{combined_content}"
            ),
        ]

        response = summarizer.invoke(messages)
        return str(response.content)
    except Exception as e:
        logger.error(f"Error summarizing memories: {e}")
        # Fall back to just using the combined content
        return combined_content


workflow = StateGraph(RuntimeState)

workflow.add_node("retrieve_memories", retrieve_relevant_memories)
workflow.add_node("respond", respond_to_user)

workflow.add_edge(START, "retrieve_memories")
workflow.add_edge("retrieve_memories", "respond")
workflow.add_edge("respond", END)

# Compile the graph
graph = workflow.compile(checkpointer=redis_saver)


def memory_consolidation_worker(user_id: str):
    """Worker that periodically consolidates memories"""
    while True:
        try:
            consolidate_memories(user_id)
            # Run every 10 minutes
            time.sleep(10 * 60)
        except Exception as e:
            logger.exception(f"Error in memory consolidation worker: {e}")
            # If there's an error, wait an hour and try again
            time.sleep(60 * 60)


def main():
    """Main interaction loop for the travel agent"""
    print("Welcome to the Travel Assistant! (Type 'exit' to quit)")
    user_id = "demo_user"

    config = RunnableConfig(
        configurable={"thread_id": "book_flight", "user_id": user_id}
    )
    state = RuntimeState(messages=[])

    # Create a queue for memory processing
    memory_queue = Queue()

    # Start a worker thread that will process memory extraction tasks
    memory_thread = threading.Thread(
        target=memory_worker, args=(memory_queue, user_id), daemon=True
    )
    memory_thread.start()

    # Start the memory consolidation thread
    consolidation_thread = threading.Thread(
        target=memory_consolidation_worker, args=(user_id,), daemon=True
    )
    consolidation_thread.start()

    # Pre-seed some knowledge
    store_memory(
        content="""
        Popular tourist destinations include Paris, Tokyo, New York, and Rome
        """,
        memory_type=MemoryType.SEMANTIC,
        user_id=user_id,
        metadata='{"category": "destinations"}',
    )

    store_memory(
        content="""
        When booking flights, always check for layover duration - at least 1
        hour for domestic and 2 hours for international flights is recommended
        """,
        memory_type=MemoryType.PROCEDURAL,
        user_id=user_id,
        metadata='{"category": "booking_tips"}',
    )

    while True:
        user_input = input("\nYou: ")

        if not user_input:
            continue

        if user_input.lower() in ["exit", "quit"]:
            print("Thank you for using the Travel Assistant. Goodbye!")
            break

        state["messages"].append(HumanMessage(content=user_input))

        try:
            # Process the user input through the graph
            final_state = None
            print(f"Incoming state: {len(state['messages'])}")
            for result in graph.stream(state, config=config, stream_mode="values"):
                final_state = RuntimeState(**result)

            # Use the final state to get and print the assistant's response once
            if final_state and final_state["messages"]:
                logger.debug(f"Len of messages: {len(final_state['messages'])}")
                logger.debug(f"Final messages: {len(final_state['messages'])}")

                assistant_message = final_state["messages"][-1]
                if isinstance(assistant_message, AIMessage):
                    print(f"\nA: {assistant_message.content}")

                summarized_state = summarize_conversation(final_state, config)
                if summarized_state:
                    logger.debug(
                        f"Using new summarized state: {len(summarized_state['messages'])}"
                    )
                    final_state = summarized_state
                    graph.update_state(
                        config, values=final_state, as_node="retrieve_memories"
                    )

                state = final_state

            # Add the current state to the memory processing queue
            memory_queue.put((state.copy(), config))

        except Exception as e:
            logger.exception(f"Error processing request: {e}")
            print(
                "\nAssistant: I'm sorry, I encountered an error processing your request."
            )


if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'langchain_redis'